In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from preprocess_data import *

In [20]:
path='./data/'

users = pd.read_csv(path + 'users_preprocessed.csv')
books = pd.read_csv(path + 'books_merged.csv')
train = pd.read_csv(path + 'train_ratings.csv')
test = pd.read_csv(path + 'test_ratings.csv')
sub = pd.read_csv(path + 'sample_submission.csv')

In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           68092 non-null  int64  
 1   age               67931 non-null  float64
 2   location_city     68079 non-null  object 
 3   location_state    67950 non-null  object 
 4   location_country  67920 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.6+ MB


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  306795 non-null  int64 
 1   isbn     306795 non-null  object
 2   rating   306795 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.0+ MB


In [7]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   year_of_publication  149570 non-null  float64
 3   publisher            149570 non-null  object 
 4   img_url              149570 non-null  object 
 5   language             82343 non-null   object 
 6   summary              82343 non-null   object 
 7   img_path             149570 non-null  object 
 8   category_high        80719 non-null   object 
 9   book_author          149570 non-null  object 
 10  category             80109 non-null   object 
 11  new_language         148470 non-null  object 
 12  remove_country_code  148470 non-null  object 
 13  book_author_over3    149570 non-null  object 
 14  book_author_over5    149570 non-null  object 
 15  book_author_over1

In [13]:
features_name = ['isbn', 'category_high', 'publisher', 'new_language', 'book_author_over10']
context_df = train.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')

In [14]:
context_df

,user_id,isbn,rating,age,location_city,location_state,location_country,category_high,publisher,new_language,book_author_over10
0,8,0002005018,4,18.0,ontario_others,ontario,canada,actresses,HarperFlamingo Canada,en,others
1,67544,0002005018,7,30.0,toronto,ontario,canada,actresses,HarperFlamingo Canada,en,others
2,123629,0002005018,8,30.2,kingston,ontario,canada,actresses,HarperFlamingo Canada,en,others
3,200273,0002005018,8,18.0,ontario_others,ontario,canada,actresses,HarperFlamingo Canada,en,others
4,210926,0002005018,9,26.5,guelph,ontario,canada,actresses,HarperFlamingo Canada,en,others
...,...,...,...,...,...,...,...,...,...,...,...
306790,278843,0743525493,7,28.0,california_others,california,usa,NaN,Simon & Schuster Audio,en,others
306791,278851,067161746X,6,33.0,dallas,texas,usa,humor,Pocket Books,en,others
306792,278851,0884159221,7,33.0,dallas,texas,usa,NaN,Lone Star Books,en,others
306793,278851,0912333022,7,33.0,dallas,texas,usa,fiction,Kqed Books,en,others


In [21]:
users = preprocess_location(preprocess_age(users))
ratings = pd.concat([train, test]).reset_index(drop=True)

print(type(features_name))

context_df = ratings.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')

<class 'list'>


In [22]:
context_df

,user_id,isbn,rating,age,location_city,location_state,location_country,category_high,publisher,new_language,book_author_over10
0,8,0002005018,4,18.000000,ontario_others,ontario,canada,actresses,HarperFlamingo Canada,en,others
1,67544,0002005018,7,30.000000,toronto,ontario,canada,actresses,HarperFlamingo Canada,en,others
2,123629,0002005018,8,30.200000,kingston,ontario,canada,actresses,HarperFlamingo Canada,en,others
3,200273,0002005018,8,18.000000,ontario_others,ontario,canada,actresses,HarperFlamingo Canada,en,others
4,210926,0002005018,9,26.500000,guelph,ontario,canada,actresses,HarperFlamingo Canada,en,others
...,...,...,...,...,...,...,...,...,...,...,...
383489,278543,1576734218,0,39.000000,california_others,california,usa,family,Multnomah,en,others
383490,278563,3492223710,0,37.000000,wien,wien,austria,others,Piper,de,others
383491,278633,1896095186,0,31.833333,sandy,utah,usa,fiction,Polestar Book Publishers,en,others
383492,278668,8408044079,0,48.000000,madrid,madrid,spain,NaN,Planeta Publishing Corporation,es,others


In [ ]:
def process_context_data(users, books, ratings1, ratings2, features_name:list):
    # publisher 전처리
    # books = preprocess_publisher(books)
    # age, location 전처리
    users = preprocess_location(preprocess_age(users))
    ratings = pd.concat([ratings1, ratings2]).reset_index(drop=True)

    print(type(features_name))
    
    # 인덱싱 처리된 데이터 조인
    """
    users_preprocessed:
        dataframe
        user_id,age,location_city,location_state,location_country
        
    books_merged:
        dataframe
        isbn,book_title,year_of_publication,publisher,img_url,
        language,summary,img_path,category_high,book_author,category,
        new_language,remove_country_code,book_author_over3,book_author_over5,
        book_author_over10,book_author_over50,book_author_over100
    """
    # user_id, isbn, age, city, state, country, category_high, publisher_4_digit, language, author_10
    # context_df = ratings.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
    context_df = ratings.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
    train_df = ratings1.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
    test_df = ratings2.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')

    idx = dict()
    # users 인덱싱
    idx, train_df, test_df = users2idx(context_df, train_df, test_df, idx)
    
    # books 인덱싱
    idx, train_df, test_df = books2idx(context_df, train_df, test_df, features_name, idx)
    
    return idx, train_df, test_df